# CS 521 - HW 1

Code

In [1]:
!pip install tensorboardX numpy torch torchvision matplotlib tqdm wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 12848026.06it/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 343370.71it/s]


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3205110.55it/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6420805.11it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [10]:
# The last argument 'targeted' can be used to toggle between a targeted and untargeted attack.
def fgsm(model, x, y, eps):
    model.eval()
    model.zero_grad()

    x_cp = x.clone().detach().requires_grad_(True)

    with torch.enable_grad():
        out = model(x_cp)
        loss = torch.nn.functional.cross_entropy(out, y)
        loss.backward()
        x_adv = x_cp + eps * x_cp.grad.sign()
        x_adv = torch.clamp(x_adv, 0, 1)
    return x_adv.detach()



def pgd_untargeted(model, x, y, *, k=7, eps=0.1, eps_step=0.025):
    model.eval()
    x_adv = x.clone().detach()
    x_orig = x.clone().detach()

    for _ in range(k):
        x_adv.requires_grad_(True)
        outputs = model(x_adv)
        loss = F.cross_entropy(outputs, y)
        model.zero_grad()
        loss.backward()
        grad_sign = x_adv.grad.sign()
        x_adv = x_adv + eps_step * grad_sign
        eta = torch.clamp(x_adv - x_orig, min=-eps, max=eps)
        x_adv = x_orig + eta
        x_adv = torch.clamp(x_adv, 0, 1).detach()

    return x_adv

# Implement Adversarial Training

In [4]:
def train_model(model, num_epochs, enable_defense=True, attack='pgd', eps=0.1):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            # Generate adversarial examples if defense is enabled
            if enable_defense:
                if attack == 'fgsm':
                    data_adv = fgsm(model, data, target, eps=eps)
                elif attack == 'pgd':
                    data_adv = pgd_untargeted(model, data, target, k=7, eps=eps, eps_step=eps/4)
                else:
                    data_adv = data
                output = model(data_adv)
            else:
                output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            total_correct += pred.eq(target).sum().item()
        avg_loss = total_loss / len(train_loader)
        accuracy = 100. * total_correct / len(train_dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')


In [5]:
def test_model_on_attacks(model, attack='pgd', eps=0.1):
    # TODO: implement this function to test the robust accuracy of the given model
    # use pgd_untargeted() within this function
    model.eval()
    total_correct = 0
    total_adv_correct = 0
    total = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # Standard prediction
        output = model(data)
        pred = output.argmax(dim=1)
        total_correct += pred.eq(target).sum().item()
        # Generate adversarial examples
        if attack == 'fgsm':
            data_adv = fgsm(model, data, target, eps=eps)
        elif attack == 'pgd':
            data_adv = pgd_untargeted(model, data, target, k=7, eps=eps, eps_step=eps/4)
        else:
            data_adv = data
        # Prediction on adversarial examples
        output_adv = model(data_adv)
        pred_adv = output_adv.argmax(dim=1)
        total_adv_correct += pred_adv.eq(target).sum().item()
        total += target.size(0)
    standard_accuracy = 100. * total_correct / total
    adv_accuracy = 100. * total_adv_correct / total
    print(f'Standard Accuracy: {standard_accuracy:.2f}%')
    print(f'Accuracy under attack ({attack}, eps={eps}): {adv_accuracy:.2f}%')



# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.

In [6]:
## train the original model
model = nn.Sequential(Normalize(), Net())
model = model.to(device)
model.train()

train_model(model, 20, False)
torch.save(model.state_dict(), 'weights.pt')

Epoch 1/20, Loss: 0.2394, Accuracy: 92.93%
Epoch 2/20, Loss: 0.1002, Accuracy: 96.90%
Epoch 3/20, Loss: 0.0680, Accuracy: 97.90%
Epoch 4/20, Loss: 0.0509, Accuracy: 98.36%
Epoch 5/20, Loss: 0.0391, Accuracy: 98.75%
Epoch 6/20, Loss: 0.0325, Accuracy: 98.92%
Epoch 7/20, Loss: 0.0255, Accuracy: 99.18%
Epoch 8/20, Loss: 0.0230, Accuracy: 99.22%
Epoch 9/20, Loss: 0.0190, Accuracy: 99.34%
Epoch 10/20, Loss: 0.0176, Accuracy: 99.41%
Epoch 11/20, Loss: 0.0148, Accuracy: 99.49%
Epoch 12/20, Loss: 0.0126, Accuracy: 99.54%
Epoch 13/20, Loss: 0.0150, Accuracy: 99.46%
Epoch 14/20, Loss: 0.0129, Accuracy: 99.57%
Epoch 15/20, Loss: 0.0106, Accuracy: 99.62%
Epoch 16/20, Loss: 0.0135, Accuracy: 99.55%
Epoch 17/20, Loss: 0.0102, Accuracy: 99.66%
Epoch 18/20, Loss: 0.0061, Accuracy: 99.80%
Epoch 19/20, Loss: 0.0129, Accuracy: 99.56%
Epoch 20/20, Loss: 0.0082, Accuracy: 99.74%


In [7]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
## PGD attack
model = nn.Sequential(Normalize(), Net())
model.load_state_dict(torch.load('weights.pt'))

for eps in [0.05, 0.1, 0.15, 0.2]:
    print(f"\nTesting with epsilon = {eps}")
    test_model_on_attacks(model, attack='pgd', eps=eps)


Testing with epsilon = 0.05
Standard Accuracy: 98.12%
Accuracy under attack (pgd, eps=0.05): 76.32%

Testing with epsilon = 0.1
Standard Accuracy: 98.12%
Accuracy under attack (pgd, eps=0.1): 22.37%

Testing with epsilon = 0.15
Standard Accuracy: 98.12%
Accuracy under attack (pgd, eps=0.15): 3.95%

Testing with epsilon = 0.2
Standard Accuracy: 98.12%
Accuracy under attack (pgd, eps=0.2): 0.56%


In [9]:
## PGD based adversarial training
model = nn.Sequential(Normalize(), Net())
eps = 0.1
train_model(model, 20, True, 'pgd', eps)
torch.save(model.state_dict(), f'weights_AT_{eps}.pt')

Epoch 1/20, Loss: 0.7839, Accuracy: 73.61%
Epoch 2/20, Loss: 0.4959, Accuracy: 83.64%
Epoch 3/20, Loss: 0.4362, Accuracy: 85.69%
Epoch 4/20, Loss: 0.4029, Accuracy: 86.81%
Epoch 5/20, Loss: 0.3838, Accuracy: 87.33%
Epoch 6/20, Loss: 0.3686, Accuracy: 87.90%
Epoch 7/20, Loss: 0.3570, Accuracy: 88.28%
Epoch 8/20, Loss: 0.3485, Accuracy: 88.48%
Epoch 9/20, Loss: 0.3394, Accuracy: 88.78%
Epoch 10/20, Loss: 0.3329, Accuracy: 89.06%
Epoch 11/20, Loss: 0.3294, Accuracy: 89.14%
Epoch 12/20, Loss: 0.3235, Accuracy: 89.26%
Epoch 13/20, Loss: 0.3208, Accuracy: 89.32%
Epoch 14/20, Loss: 0.3170, Accuracy: 89.48%
Epoch 15/20, Loss: 0.3155, Accuracy: 89.64%
Epoch 16/20, Loss: 0.3111, Accuracy: 89.72%
Epoch 17/20, Loss: 0.3084, Accuracy: 89.83%
Epoch 18/20, Loss: 0.3050, Accuracy: 89.94%
Epoch 19/20, Loss: 0.3019, Accuracy: 89.98%
Epoch 20/20, Loss: 0.2993, Accuracy: 90.11%


In [11]:
## fgsm attack
model = nn.Sequential(Normalize(), Net())
model.load_state_dict(torch.load('weights_AT_0.1.pt'))

for eps in [0.05, 0.1, 0.15, 0.2]:
    test_model_on_attacks(model, attack='fgsm', eps=eps)

Standard Accuracy: 98.25%
Accuracy under attack (fgsm, eps=0.05): 95.63%
Standard Accuracy: 98.25%
Accuracy under attack (fgsm, eps=0.1): 90.54%
Standard Accuracy: 98.25%
Accuracy under attack (fgsm, eps=0.15): 80.38%
Standard Accuracy: 98.25%
Accuracy under attack (fgsm, eps=0.2): 61.75%


In [12]:
## fgsm based adversarial training
model = nn.Sequential(Normalize(), Net())
eps = 0.1
train_model(model, 20, True, 'fgsm', eps)
torch.save(model.state_dict(), f'weights_AT_{eps}.pt')

Epoch 1/20, Loss: 0.7563, Accuracy: 74.62%
Epoch 2/20, Loss: 0.4660, Accuracy: 84.69%
Epoch 3/20, Loss: 0.4048, Accuracy: 86.89%
Epoch 4/20, Loss: 0.3737, Accuracy: 87.73%
Epoch 5/20, Loss: 0.3518, Accuracy: 88.32%
Epoch 6/20, Loss: 0.3367, Accuracy: 88.88%
Epoch 7/20, Loss: 0.3268, Accuracy: 89.27%
Epoch 8/20, Loss: 0.3133, Accuracy: 89.55%
Epoch 9/20, Loss: 0.3073, Accuracy: 89.85%
Epoch 10/20, Loss: 0.2995, Accuracy: 90.06%
Epoch 11/20, Loss: 0.2927, Accuracy: 90.19%
Epoch 12/20, Loss: 0.2890, Accuracy: 90.31%
Epoch 13/20, Loss: 0.2844, Accuracy: 90.46%
Epoch 14/20, Loss: 0.2788, Accuracy: 90.66%
Epoch 15/20, Loss: 0.2729, Accuracy: 90.88%
Epoch 16/20, Loss: 0.2701, Accuracy: 91.05%
Epoch 17/20, Loss: 0.2658, Accuracy: 91.06%
Epoch 18/20, Loss: 0.2636, Accuracy: 91.28%
Epoch 19/20, Loss: 0.2609, Accuracy: 91.22%
Epoch 20/20, Loss: 0.2605, Accuracy: 91.27%
